In [1]:
!pip uninstall -y numpy
!pip install numpy==1.23.5 gensim==4.3.1 --quiet

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 

In [2]:
import pandas as pd
import re
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Baca file CSV
df = pd.read_csv("ulasan_aplikasi_traveloka.csv")
df.head()

,content
0,mending gausah ada fitur check in online kalo ...
1,tolong perbaiki permasalahan harga tiket yang ...
2,"Aplikasi bagus, namun terkadang tidak jujur, s..."
3,Sudah beberapa kali saya memesan tiket pesawat...
4,Traveloka ga jelas masa saya refund 2 hotel be...


In [4]:
df.dropna(inplace=True)
clean_df = df
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130482 entries, 0 to 130499
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   content  130482 non-null  object
dtypes: object(1)
memory usage: 2.0+ MB


In [5]:
# preprocesing text
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in text]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [6]:
import requests

def download_slangwords(url):
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()
    slang_dict = {}
    for line in lines:
        parts = line.split()
        if len(parts) == 2:
            slang_dict[parts[0].lower()] = parts[1]
    return slang_dict

url = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'

slangwords = download_slangwords(url)

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [7]:
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

clean_df

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,mending gausah ada fitur check in online kalo ...,mending gausah ada fitur check in online kalo ...,mending gausah ada fitur check in online kalo ...,mending gausah ada fitur check in online kalo ...,"[mending, gausah, ada, fitur, check, in, onlin...","[mending, gausah, fitur, check, online, kalo, ...",mending gausah fitur check online kalo sistemn...
1,tolong perbaiki permasalahan harga tiket yang ...,tolong perbaiki permasalahan harga tiket yang ...,tolong perbaiki permasalahan harga tiket yang ...,tolong perbaiki permasalahan harga tiket yang ...,"[tolong, perbaiki, permasalahan, harga, tiket,...","[tolong, perbaiki, permasalahan, harga, tiket,...",tolong perbaiki permasalahan harga tiket keter...
2,"Aplikasi bagus, namun terkadang tidak jujur, s...",Aplikasi bagus namun terkadang tidak jujur saa...,aplikasi bagus namun terkadang tidak jujur saa...,aplikasi bagus namun terkadang tidak jujur saa...,"[aplikasi, bagus, namun, terkadang, tidak, juj...","[aplikasi, bagus, terkadang, jujur, pembayaran...",aplikasi bagus terkadang jujur pembayaran kali...
3,Sudah beberapa kali saya memesan tiket pesawat...,Sudah beberapa kali saya memesan tiket pesawat...,sudah beberapa kali saya memesan tiket pesawat...,sudah beberapa kali saya memesan tiket pesawat...,"[sudah, beberapa, kali, saya, memesan, tiket, ...","[kali, memesan, tiket, pesawat, pax, harganya,...",kali memesan tiket pesawat pax harganya mahal ...
4,Traveloka ga jelas masa saya refund 2 hotel be...,Traveloka ga jelas masa saya refund hotel ber...,traveloka ga jelas masa saya refund hotel ber...,traveloka ga jelas masa saya refund hotel berh...,"[traveloka, ga, jelas, masa, saya, refund, hot...","[traveloka, refund, hotel, berhasil, tiba², mu...",traveloka refund hotel berhasil tiba² muncul t...
...,...,...,...,...,...,...,...
130495,Terbantu tapi soal pembayaran via atm tdk 24 jam,Terbantu tapi soal pembayaran via atm tdk jam,terbantu tapi soal pembayaran via atm tdk jam,terbantu tapi soal pembayaran via atm tdk jam,"[terbantu, tapi, soal, pembayaran, via, atm, t...","[terbantu, pembayaran, via, atm, tdk, jam]",terbantu pembayaran via atm tdk jam
130496,Sangat memudahkan dalam urusan perjalanan via ...,Sangat memudahkan dalam urusan perjalanan via ...,sangat memudahkan dalam urusan perjalanan via ...,sangat memudahkan dalam urusan perjalanan via ...,"[sangat, memudahkan, dalam, urusan, perjalanan...","[memudahkan, urusan, perjalanan, via, air, plane]",memudahkan urusan perjalanan via air plane
130497,Kalau mw berangkat tinggal pesan tiket aj hahae,Kalau mw berangkat tinggal pesan tiket aj hahae,kalau mw berangkat tinggal pesan tiket aj hahae,kalau mw berangkat tinggal pesan tiket aj hahae,"[kalau, mw, berangkat, tinggal, pesan, tiket, ...","[mw, berangkat, tinggal, pesan, tiket, aj, hahae]",mw berangkat tinggal pesan tiket aj hahae
130498,"Mantap,jujur,fair! Like this.",Mantapjujurfair Like this,mantapjujurfair like this,mantapjujurfair like this,"[mantapjujurfair, like, this]","[mantapjujurfair, like]",mantapjujurfair like


In [8]:
import csv
import requests
from io import StringIO

lexicon_positive = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

lexicon_negative = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch negative lexicon data")


In [9]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0

    for word in text:
        if word in lexicon_positive:
            score = score + lexicon_positive[word]

    for word in text:
        if word in lexicon_negative:
            score = score + lexicon_negative[word]

    polarity = ''
    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity


In [10]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    63109
neutral     37075
negative    30298
Name: count, dtype: int64


**Pelatihan: LSTM, Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20**

In [17]:
clean_df.head()

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,mending gausah ada fitur check in online kalo ...,mending gausah ada fitur check in online kalo ...,mending gausah ada fitur check in online kalo ...,mending gausah ada fitur check in online kalo ...,"[mending, gausah, ada, fitur, check, in, onlin...","[mending, gausah, fitur, check, online, kalo, ...",mending gausah fitur check online kalo sistemn...,-13,negative
1,tolong perbaiki permasalahan harga tiket yang ...,tolong perbaiki permasalahan harga tiket yang ...,tolong perbaiki permasalahan harga tiket yang ...,tolong perbaiki permasalahan harga tiket yang ...,"[tolong, perbaiki, permasalahan, harga, tiket,...","[tolong, perbaiki, permasalahan, harga, tiket,...",tolong perbaiki permasalahan harga tiket keter...,3,positive
2,"Aplikasi bagus, namun terkadang tidak jujur, s...",Aplikasi bagus namun terkadang tidak jujur saa...,aplikasi bagus namun terkadang tidak jujur saa...,aplikasi bagus namun terkadang tidak jujur saa...,"[aplikasi, bagus, namun, terkadang, tidak, juj...","[aplikasi, bagus, terkadang, jujur, pembayaran...",aplikasi bagus terkadang jujur pembayaran kali...,9,positive
3,Sudah beberapa kali saya memesan tiket pesawat...,Sudah beberapa kali saya memesan tiket pesawat...,sudah beberapa kali saya memesan tiket pesawat...,sudah beberapa kali saya memesan tiket pesawat...,"[sudah, beberapa, kali, saya, memesan, tiket, ...","[kali, memesan, tiket, pesawat, pax, harganya,...",kali memesan tiket pesawat pax harganya mahal ...,3,positive
4,Traveloka ga jelas masa saya refund 2 hotel be...,Traveloka ga jelas masa saya refund hotel ber...,traveloka ga jelas masa saya refund hotel ber...,traveloka ga jelas masa saya refund hotel berh...,"[traveloka, ga, jelas, masa, saya, refund, hot...","[traveloka, refund, hotel, berhasil, tiba², mu...",traveloka refund hotel berhasil tiba² muncul t...,6,positive


In [11]:
!pip install gensim

### Pelatihan: SVM, Ekstraksi Fitur: TF-IDF, Pembagian Data: 80/20

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X = clean_df['text_akhir']
y = clean_df['polarity']

tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

svm = SVC()
svm.fit(X_train, y_train)

y_pred_train_svm = svm.predict(X_train)
y_pred_test_svm = svm.predict(X_test)

svm_tfidf_train_accuracy = accuracy_score(y_train, y_pred_train_svm)
svm_tfidf_test_accuracy = accuracy_score(y_test, y_pred_test_svm)

print("SVM - accuracy_train:", svm_tfidf_train_accuracy)
print("SVM - accuracy_test:", svm_tfidf_test_accuracy)

SVM - accuracy_train: 0.9105331225750827
SVM - accuracy_test: 0.8863854082844771


### Pelatihan: Random Forest (RF), Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20

In [34]:
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier

# Tokenisasi teks
tokenized = [text.split() for text in clean_df['text_akhir'].astype(str)]

# Latih model Word2Vec
w2v_model = Word2Vec(sentences=tokenized, vector_size=100, window=5, min_count=1, workers=4)

# Rata-rata vektor kata untuk setiap dokumen
def vector_average(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

X_vec = np.array([vector_average(tokens) for tokens in tokenized])
y = clean_df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf = rf.predict(X_test)

rf_w2v_train_accuracy = accuracy_score(y_train, y_pred_train_rf)
rf_w2v_test_accuracy = accuracy_score(y_test, y_pred_test_rf)

print("Random Forest (Word2Vec) - accuracy_train:", rf_w2v_train_accuracy)
print("Random Forest (Word2Vec) - accuracy_test:", rf_w2v_test_accuracy)

Random Forest (Word2Vec) - accuracy_train: 1.0
Random Forest (Word2Vec) - accuracy_test: 0.879373108020079


### Pelatihan: Random Forest (RF), Ekstraksi Fitur: TF-IDF, Pembagian Data: 70/30

In [36]:
from sklearn.ensemble import RandomForestClassifier

X = clean_df['text_akhir']
y = clean_df['polarity']

tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred_train_rf2 = rf.predict(X_train)
y_pred_test_rf2 = rf.predict(X_test)

rf_tfidf_train_accuracy = accuracy_score(y_train, y_pred_train_rf2)
rf_tfidf_test_accuracy = accuracy_score(y_test, y_pred_test_rf2)

print("Random Forest (TF-IDF) - accuracy_train:", rf_tfidf_train_accuracy)
print("Random Forest (TF-IDF) - accuracy_test:", rf_tfidf_test_accuracy)


Random Forest (TF-IDF) - accuracy_train: 0.9429803912981596
Random Forest (TF-IDF) - accuracy_test: 0.8757184825648231


### Deep Learning untuk Teks dengan Tokenizer + Embedding + LSTM

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Data dan label
X = clean_df['text_akhir']
y = clean_df['polarity']

# Encode label ke one-hot
label_encoder = LabelEncoder()
y_encoded = to_categorical(label_encoder.fit_transform(y))

# Tokenisasi teks
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

maxlen = 100
X_padded = pad_sequences(X_sequences, maxlen=maxlen, padding='post', truncating='post')

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Bangun model LSTM
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Compile dan fit model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluasi
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print("LSTM Tokenizer - accuracy_train:", train_acc)
print("LSTM Tokenizer - accuracy_test:", test_acc)

Epoch 1/5
3263/3263 ━━━━━━━━━━━━━━━━━━━━ 211s 63ms/step - accuracy: 0.4839 - loss: 1.0500 - val_accuracy: 0.4801 - val_loss: 1.0510
Epoch 2/5
3263/3263 ━━━━━━━━━━━━━━━━━━━━ 238s 56ms/step - accuracy: 0.4832 - loss: 1.0490 - val_accuracy: 0.4801 - val_loss: 1.0508
Epoch 3/5
3263/3263 ━━━━━━━━━━━━━━━━━━━━ 180s 55ms/step - accuracy: 0.4822 - loss: 1.0491 - val_accuracy: 0.4801 - val_loss: 1.0510
Epoch 4/5
3263/3263 ━━━━━━━━━━━━━━━━━━━━ 202s 55ms/step - accuracy: 0.4813 - loss: 1.0494 - val_accuracy: 0.4801 - val_loss: 1.0510
Epoch 5/5
3263/3263 ━━━━━━━━━━━━━━━━━━━━ 212s 59ms/step - accuracy: 0.4812 - loss: 1.0494 - val_accuracy: 0.4801 - val_loss: 1.0511
3263/3263 ━━━━━━━━━━━━━━━━━━━━ 51s 16ms/step - accuracy: 0.4867 - loss: 1.0459
816/816 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.4838 - loss: 1.0483
LSTM Tokenizer - accuracy_train: 0.4845811128616333
LSTM Tokenizer - accuracy_test: 0.48009350895881653


In [41]:
import pandas as pd

results_df = pd.DataFrame({
    'Model': [
        'SVM (TF-IDF, 80/20)',
        'Random Forest (Word2Vec, 80/20)',
        'Random Forest (TF-IDF, 70/30)',
        'LSTM (Tokenizer, Embedding, LSTM)'
    ],
    'Accuracy Train': [
        svm_tfidf_train_accuracy,
        rf_w2v_train_accuracy,
        rf_tfidf_train_accuracy,
        train_acc
    ],
    'Accuracy Test': [
        svm_tfidf_test_accuracy,
        rf_w2v_test_accuracy,
        rf_tfidf_test_accuracy,
        test_acc
    ]
})

results_df


,Model,Accuracy Train,Accuracy Test
0,"SVM (TF-IDF, 80/20)",0.910533,0.886385
1,"Random Forest (Word2Vec, 80/20)",1.000000,0.879373
2,"Random Forest (TF-IDF, 70/30)",0.942980,0.875718
3,"LSTM (Tokenizer, Embedding, LSTM)",0.484581,0.480094


Dalam kasus ini, model dengan pelatihan SVM dengan ekstrasi fitur TF-IDF akan saya pilih karena dari hasil accuracy train yang mencapai 91% dan accuracy test mencapai 88% dinilai lebih balanced untuk dilakukan pada tahap inference atau testing. Untuk model yang menggunakan Random Forest (Word2Vec, 80/20), Random Forest (TF-IDF, 70/30) atau LSTM (Tokenizer, Embedding, LSTM)	masing-masing memiliki kekurangannya. Model Random Forest dengan Word2Vec memang menunjukkan akurasi pelatihan sempurna (100%), namun hal ini mengindikasikan kemungkinan overfitting karena akurasi pengujiannya lebih rendah dibanding SVM. Sementara itu, model Random Forest dengan TF-IDF (70/30) memiliki akurasi yang cukup baik, tetapi proporsi data latih dan uji yang berbeda dapat memengaruhi stabilitas model. Adapun model LSTM menunjukkan akurasi yang paling rendah, baik pada pelatihan maupun pengujian, sehingga kurang direkomendasikan untuk tahap inference dalam kasus ini.

In [47]:
# Input kalimat baru dari pengguna
kalimat_baru = input("Masukkan kalimat baru: ")

# Melakukan preprocessing pada kalimat baru
kalimat_baru_cleaned = cleaningText(kalimat_baru)
kalimat_baru_casefolded = casefoldingText(kalimat_baru_cleaned)
kalimat_baru_slangfixed = fix_slangwords(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenizingText(kalimat_baru_slangfixed)
kalimat_baru_filtered = filteringText(kalimat_baru_tokenized)
kalimat_baru_final = toSentence(kalimat_baru_filtered)

# Transformasi dengan TF-IDF (menggunakan objek tfidf dari pelatihan)
X_kalimat_baru = tfidf.transform([kalimat_baru_final])

# Prediksi menggunakan model SVM yang sudah dilatih
prediksi_sentimen = svm.predict(X_kalimat_baru)

# Menampilkan hasil prediksi
if prediksi_sentimen[0] == 'positive':
    print("Sentimen kalimat baru adalah POSITIF.")
elif prediksi_sentimen[0] == 'negative':
    print("Sentimen kalimat baru adalah NEGATIF.")
else:
    print("Sentimen kalimat baru adalah NETRAL.")

Masukkan kalimat baru: Jelek sekali sekarang aplikasinya, setiap saya mau cari tiket pasti apps error
Sentimen kalimat baru adalah NEGATIF.
